In [13]:
from sklearn.model_selection import train_test_split
from pyspark.sql import SparkSession
import pandas as pd
from pandas.api.types import is_numeric_dtype as is_num
import numpy as np

In [4]:
spark = SparkSession.builder.appName("AIProject").config("spark.driver.memory", "8g").getOrCreate()
data_frame = spark.read.csv('./datasets/training_dataset.csv', header=True, inferSchema=True)
df = data_frame.limit(50000).toPandas()

In [ ]:
df = df.dropna()

df['store_and_fwd_flag'].replace(['N', 'Y'], [1, 2], inplace=True)

# df = df.drop('ID', axis=1)
# df = df.drop('tpep_pickup_datetime', axis=1)
# df = df.drop('tpep_dropoff_datetime', axis=1)

df

C:\Users\William\AppData\Local\Temp\ipykernel_26000\1658240288.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['store_and_fwd_flag'].replace(['N', 'Y'], [1, 2], inplace=True)


,vendorid,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,1.0,4.16,1.0,1,144,142,1,31.0,1.0,0.5,6.00,0.0,1.0,42.00,2.5,0.0,2116.0
1,2,1.0,1.03,1.0,1,144,79,1,10.0,1.0,0.5,2.00,0.0,1.0,17.00,2.5,0.0,549.0
2,2,2.0,0.62,1.0,1,230,186,2,-5.8,0.0,-0.5,0.00,0.0,-1.0,-9.80,-2.5,0.0,253.0
3,2,1.0,1.41,1.0,1,113,4,1,12.1,1.0,0.5,2.00,0.0,1.0,19.10,2.5,0.0,769.0
4,2,1.0,2.06,1.0,1,161,237,1,17.0,0.0,0.5,4.20,0.0,1.0,25.20,2.5,0.0,1110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2,1.0,2.50,1.0,1,236,143,1,17.7,2.5,0.5,2.00,0.0,1.0,26.20,2.5,0.0,1166.0
49996,2,1.0,0.62,1.0,1,262,263,2,7.2,0.0,0.5,0.00,0.0,1.0,11.20,2.5,0.0,388.0
49997,1,1.0,1.90,1.0,1,237,164,2,21.2,2.5,0.5,0.00,0.0,1.0,25.20,2.5,0.0,1623.0
49998,2,1.0,1.24,1.0,1,162,170,1,14.9,0.0,0.5,2.00,0.0,1.0,20.90,2.5,0.0,987.0


In [21]:
numeric_columns = [
    "vendorid", "passenger_count", "trip_distance", "pulocationid", "dolocationid",
    "ratecodeid", "payment_type", "fare_amount", "extra", "mta_tax", "improvement_surcharge",
    "tip_amount", "tolls_amount", "total_amount", "congestion_surcharge", "airport_fee", "duration"
]

for col in numeric_columns:
    assert is_num(df[col])

In [ ]:
x = df[]
y = df[]

x_train, x_test, y_train, y_test = train_test_split()